## Import modules

In [1]:
import sys
sys.path.append('src')

In [2]:
import pandas
import numpy

from energiapy.components import *
from energiapy.utils.data_utils import make_henry_price_df, remove_outliers
from itertools import product

## Data Import

The following data is needed for the model

- solar and wind profiles : energiapy.fetch_nsrdb_data imports data from the NREL NSRDB database
- power demand : ERCOT for Houston; CAISO for San Diego
- Natural Gas prices: Henry Hub Price Index for both


**Get Weather data**

In [3]:
def load_data(loc:str, index:list):
    df = pandas.read_csv(f'data/{loc}_solar19.csv')
    df['idx'] = index
    df = df.set_index('idx')
    return df

In [4]:
idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]


In [5]:
weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]


**Demand data for San Diego (CAISO for SDGE region) and Houston (ERCOT for COAST region)**

In [6]:
demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
demand_ny = pandas.DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga

**Natural gas prices from Henry Hub Price Index**  

We use the special function energiapy.make_henry_price_df because of special requirements such as filling in empty data points (weekends, public holidays). Such function will be generalized in the next update of energipy

In [7]:
ng_price = make_henry_price_df(
    file_name='data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
ng_price = ng_price.drop(columns= 'scales')
ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
ng_price = pandas.concat([ng_price])

## Define temporal scale


The variabilities of energy systems are best captured over a discretized spatio-temporal scale. In energiapy, the first declaration is the temporal scale. 

For e.g.: Here we declare three temporal scales at different levels from right to left. The interpretation of these scales is merely symentic. Scales can be declared as the problem demands.
- 0, annual, with 1 discretization
- 1, daily with 365 discretization
- 2, hourly with 24 discretization

In essence, we are creating a temporal scale of 8760 points.

In [8]:
scales = TemporalScale(discretization_list=[1, 365, 24], design_scale= 0, scheduling_scale=2)

In [9]:
scales.list

[0, 1, 2]

In [9]:
scales.ctype

[<ScaleType.MULTI: 1>, <ScaleType.SCHEDULING: 4>]

## Declare resources

Resources can be consumed, produced, stored, discharged (or sold). 

In [10]:
th_h2 = Theta((0, 10))

In [11]:
Hydrogen = Resource(name='H2', discharge = True, sell_price = Theta((0, 10)))

In [12]:
Hydrogen.sell_price

Theta(H2,sell_price)

In [13]:
Solar = Resource(name='Solar', gwp =50, eutt = 60, consume=BigM, basis='MW', label='Solar Power')

Wind = Resource(name='Wind', consume=True, basis='MW', label='Wind Power')

Power = Resource(name='Power', basis='MW', label='Power generated')

Uranium = Resource(name='Ur', consume=BigM, purchase_price= 42.70/(250/2), basis='kg', label='Uranium')

H2 = Resource(name='H2', basis='tons', label='Hydrogen')

H2_L = Resource(name='H2_L', store_max = True, basis='tons', label='Hydrogen')

CO2_AQoff = Resource(name='CO2(Aq)', store_max = True, basis='tons', label='Carbon dioxide - sequestered')

H2O = Resource(name='H2O', consume=BigM,
               purchase_price= 0.001, basis='tons', label='Water')

CH4 = Resource(name='CH4', consume=BigM, purchase_price=1, basis='tons', label='Natural gas')

CO2 = Resource(name='CO2', basis='tons', label='Carbon dioxide', block='Resource')

CO2_Vent = Resource(name='CO2_Vent', basis='tons', label='Carbon dioxide - Vented', sell_price= 20)

O2 = Resource(name='O2', discharge=True, basis='tons', label='Oxygen')

CO2_DAC = Resource(name='CO2_DAC', basis='tons', label='Carbon dioxide - captured')


In [14]:
type(BigM)

energiapy.components.parameters.special.Big

## Declare processes

In [15]:
LiI = Process(name='LiI', storage= Power, capex = 3516428, fopex= 87910, vopex = 0, store_max = 10000, cap_max=1000, label='Lithium-ion battery', basis = 'MW')

# WF = Process(name='WF', conversion={Wind: -1, Power: 1}, capex= (0, 20), fopex= (0, 3), vopex=(0, 1), cap_max= Theta((1, 2)), land = Theta((4, 10)), label='Wind mill array', basis = 'MW')

WF = Process(name='WF', gwp = 50, odp = 2900, conversion={Wind: -1, Power: 1},capex=1462000, fopex=43000, vopex=4953, cap_max=1000, land = 30, label='Wind mill array', basis = 'MW')

# PV = Process(name='PV', conversion={Solar: -1, Power: 1}, capex = 1333262, fopex= 22623, cap_max=1000, label = 'Solar PV', basis = 'MW')
PV = Process(name='PV', conversion={1:{Solar: -1, Power: 0.8}, 2: {Solar: -1, Power: 1}}, capex = 1333262, fopex= 22623, cap_max=True, label = 'Solar PV', basis = 'MW')

SMRH = Process(name='SMRH', conversion={Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 1.03, CO2: 9.332}, capex =2520000, fopex = 945000, vopex = 51.5, cap_max= 1000, label='Steam methane reforming + CCUS')

NGCC = Process(name= 'NGCC', conversion = {Power: 1, CH4: -0.108, CO2_Vent: 0.297*0.05, CO2:0.297*0.95}, capex = 2158928, fopex= 53320, vopex = 4090, cap_max= 1000, label = 'NGCC + 95% CC')

SMR = Process(name='SMR', capex = 2400, fopex = 800, vopex = 0.03,  conversion={Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 9.4979}, cap_max=1000, label='Steam methane reforming')

H2FC = Process(name='H2FC', conversion = {H2:-0.050, Power: 1}, capex =  1.6*10**6, vopex = 3.5, cap_max = 1000, label = 'hydrogen fuel cell')

DAC = Process(name='DAC', capex = 0.02536, fopex = 0.634, conversion={Power: -0.193, H2O: -4.048, CO2_DAC: 1}, cap_max=1000, label='Direct air capture')

PSH = Process(name='PSH', storage= Power, capex = 3924781, fopex= 17820, vopex = 512.5, store_max = 10000, cap_max=1000, label='Pumped storage hydropower', basis = 'MW')

H2_L_c = Process(name='H2_L_c', conversion={Power: -0.417, H2_L: 1, H2: -1}, capex =  1.6*10**6, vopex = 3.5, cap_max= 1000, label='Hydrogen geological storage')

H2_L_d = Process(name='H2_L_d', conversion={H2_L: -1, H2: 1}, capex =  0.01, vopex = 0.001, cap_max= 1000, label='Hydrogen geological storage discharge')

DAC = Process(name='DAC', conversion={Power: -0.193, H2O: -4.048, CO2_DAC: 1}, capex = 730, fopex= 114, vopex= 3.6, cap_max=1000, label='Direct air capture')

ASMR = Process(name='ASMR', conversion={Uranium: -4.17*10**(-5), H2O: -3.364, Power: 1}, capex = 7988951, fopex= 0.04*0.730, cap_max=1000, label='Small modular reactors (SMRs)')

AWE = Process(name='AWE', land = (0,20),  conversion={Power: -1, H2: 0.019, O2: 0.7632, H2O: -0.1753}, capex = 1.1*10**6, fopex = 16918, cap_max=1000, label='Alkaline water electrolysis (AWE)', citation='Demirhan et al. 2018 AIChE paper') 

AQoff_SMR = Process(name='AQoff_SMR', conversion={Power: -1.28, CO2_AQoff: 1, CO2: -1}, capex =  5.52, vopex = 4.14, cap_max= 1000,  label='Offshore aquifer CO2 sequestration (SMR)')

In [16]:
help(Process)

Help on class Process in module energiapy.components.process:

class Process(builtins.object)
 |  Process(name: str, cap_max: Union[float, Tuple[float], energiapy.components.parameters.mpvar.Theta, bool, ForwardRef('Big')], cap_min: float = None, land: float = None, conversion: Union[Dict[Union[int, str], Dict[energiapy.components.resource.Resource, float]], Dict[energiapy.components.resource.Resource, float]] = None, material_cons: Union[Dict[Union[int, str], Dict[energiapy.components.material.Material, float]], Dict[energiapy.components.material.Material, float]] = None, capex: Union[float, dict, Tuple[float], energiapy.components.parameters.mpvar.Theta] = None, fopex: Union[float, Tuple[float], energiapy.components.parameters.mpvar.Theta] = None, vopex: Union[float, Tuple[float], energiapy.components.parameters.mpvar.Theta] = None, incidental: Union[float, Tuple[float], energiapy.components.parameters.mpvar.Theta] = None, gwp: Union[float, Tuple[float], energiapy.components.paramete

In [17]:
type(BigM)

energiapy.components.parameters.special.Big

In [18]:
process_list = {LiI, WF, PV, SMRH, SMR, DAC, AWE, H2_L_c, H2_L_d, AQoff_SMR, ASMR, PSH, NGCC}

## Declare locations

In [19]:
cc = Factor(data = pandas.DataFrame({'a': [2]}), scales= scales, apply_max_scaler= False)

houston = Location(name='HO', processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR}, credit = {PV: 2}, cap_max_localize = {WF: 0.4, AWE: 23}, 
                   credit_factor ={PV: pandas.DataFrame({'a': [2]})}, consume_localize = {Solar: 2}, demand= {Power: (0, 1)}, 
                   land_cost = 10, land_cost_factor= cc, capacity_factor={PV: weather_ho[['dni']], WF: weather_ho[['wind_speed']]}, demand_factor={
                   Power: demand_ho}, discharge_factor={Power: demand_ho}, purchase_price_factor={CH4: ng_price}, land_max= True, scales=scales, label='Houston')

sandiego = Location(name='SD', processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR}, purchase_price_localize = {CH4: 222}, 
                    consume_localize = {Solar: 3}, land_cost = 30, land_cost_factor = pandas.DataFrame({'a': [2]}) , 
                    credit = {SMRH: 30, AQoff_SMR: Theta((0, 23))}, credit_factor= {SMRH: pandas.DataFrame({'a': [1 - 0.001*i for i in range(365)]})}, 
                    capacity_factor={PV: weather_sd[['dni']], WF: weather_sd[['wind_speed']]}, demand_factor={
                    Power: demand_sd}, purchase_price_factor={CH4: ng_price}, discharge_localize= {O2: 0.2}, scales=scales, label='SanDiego')

newyork = Location(name='NY', processes={LiI, H2FC}, land_max = (0, 50000), land_max_factor = pandas.DataFrame({'a': [2]}), cap_max_localize = {PV: 0.2, AWE: 34}, 
                   capacity_factor={PV: weather_ny[['dni']], WF: weather_ny[['wind_speed']]}, demand= {Power: 40}, demand_factor={
                   Power: demand_ny}, credit = {PV: 7}, purchase_price_localize = {CH4: 2}, scales=scales, label='NewYork')


In [20]:
PV.cap_max

BigM

In [21]:
PV.factors

{'PROCESS_CREDIT': {HO: Factor(PV,HO,credit)},
 'PROCESS_CAPACITY': {HO: Factor(PV,HO,capacity),
  SD: Factor(PV,SD,capacity),
  NY: Factor(PV,NY,capacity)}}

In [22]:
houston.ptype

{<LocationParamType.LAND_MAX: 2>: <ParameterType.UNBOUNDED: 3>,
 <LocationParamType.LAND_COST: 1>: <ParameterType.CERTAIN: 1>}

In [23]:
houston.land_cost_factor

Factor(HO,land_cost)

In [24]:
# for i in houston.processes | sandiego.processes | newyork.processes:
# # for i in houston.resources | sandiego.resources | newyork.resources:
#     print(i, i.localizations)
#     print(i, i.factors)
    

In [25]:
Power.ctype

[<ResourceType.PRODUCE: 2>,
 <ResourceType.IMPLICIT: 3>,
 {<ResourceType.DEMAND: 8>: {HO, NY}},
 <ResourceType.DISCHARGE: 4>]

## Declare transports

In [26]:

# Train_H2 = Transport(name='Train_H2', resources={
#                      H2}, trans_max=bigM, trans_loss=0.03, trans_cost=1.667*10**(-3), label='Railway for hydrogen transportation')
# Grid = Transport(name='Grid', resources={
#                  Power}, trans_max=bigM, trans_loss=0.001, trans_cost=0.5*10**(-3), label='Railroad transport')


Train_H2 = Transport(name='Train_H2', resources={
                     H2}, cap_max=BigM, trans_loss=0.03, capex = 1000, vopex=1.667, fopex = (0,1), acid = 34, eutf =55, label='Railway for hydrogen transportation')
Grid = Transport(name='Grid', resources={
                 H2, Power}, cap_max=BigM, trans_loss=0.001, capex = 500, vopex = 0.444, label='Railroad transport')



In [27]:
Train_H2.cap_max

BigM

## Declare Network

In [28]:
distance_matrix = [
    [2366], 
    [2620]
]

transport_matrix = [
    [[Train_H2]], 
    [[Grid]],
]
capacity_factor  = {(houston, newyork): {Train_H2: weather_sd[['wind_speed']]}, (sandiego, newyork): {Train_H2: weather_sd[['wind_speed']]}}

network = Network(name='network', scales = scales, land_max= (0,5), land_max_factor= pandas.DataFrame({'a': [5]}),  
                  capacity_factor= capacity_factor, vopex_factor = {(houston, sandiego): {Train_H2: weather_sd[['wind_speed']]}}, sources=[houston, sandiego],
                  sinks=[newyork], distance_matrix=distance_matrix, transport_matrix=transport_matrix)


In [29]:

scenario = Scenario(name='scenario_full', network=network, scales=scales, label='full_case')


In [31]:
scenario

scenario_full

In [30]:
for i in houston.resources:
    print(i.ctype)

[<ResourceType.SELL: 5>, <ResourceType.DISCHARGE: 4>, <ResourceType.PRODUCE: 2>]
[<ResourceType.CONSUME: 6>, <ResourceType.EMISSION: 10>]
[<ResourceType.CONSUME: 6>]
[<ResourceType.PRODUCE: 2>, <ResourceType.TRANSPORT: 9>, <ResourceType.TRANSPORT: 9>]
[<ResourceType.PRODUCE: 2>, {<ResourceType.DEMAND: 8>: {HO, NY}}, <ResourceType.DISCHARGE: 4>, <ResourceType.TRANSPORT: 9>]
[<ResourceType.PRODUCE: 2>, <ResourceType.IMPLICIT: 3>, <ResourceType.STORE: 1>]
[<ResourceType.PRODUCE: 2>, <ResourceType.IMPLICIT: 3>]
[<ResourceType.CONSUME: 6>, <ResourceType.PURCHASE: 7>]
[<ResourceType.DISCHARGE: 4>, <ResourceType.PRODUCE: 2>]
[<ResourceType.CONSUME: 6>, <ResourceType.PURCHASE: 7>]
[<ResourceType.PRODUCE: 2>, <ResourceType.IMPLICIT: 3>, <ResourceType.STORE: 1>]


In [26]:
houston.resources_produce

{CO2, CO2(Aq), CO2_Vent, H2, LiI_Power_stored, O2, Power}

In [46]:
ResourceType.all()

{'CONSUME',
 'DEMAND',
 'DISCHARGE',
 'EMISSION',
 'IMPLICIT',
 'PRODUCE',
 'PURCHASE',
 'SELL',
 'STORE',
 'TRANSPORT'}

In [37]:
ResourceType.resource_level()

{'CONSUME',
 'DISCHARGE',
 'EMISSION',
 'IMPLICIT',
 'PRODUCE',
 'PURCHASE',
 'SELL',
 'STORE'}

In [36]:
ResourceType.location_level()

{'DEMAND'}

In [38]:
ResourceType.transport_level()

{'TRANSPORT'}

In [59]:
ProcessType.all()

{'CREDIT',
 'EMISSION',
 'EXPENDITURE',
 'FAILURE',
 'INTERMITTENT',
 'LAND',
 'LINEAR_CAPEX',
 'MULTI_MATMODE',
 'MULTI_PRODMODE',
 'NO_MATMODE',
 'PWL_CAPEX',
 'READINESS',
 'SINGLE_MATMODE',
 'SINGLE_PRODMODE',
 'STORAGE',
 'STORAGE_DISCHARGE',
 'STORAGE_REQ'}

In [60]:
ProcessType.process_level()

{'EMISSION',
 'EXPENDITURE',
 'FAILURE',
 'LAND',
 'LINEAR_CAPEX',
 'MULTI_MATMODE',
 'MULTI_PRODMODE',
 'NO_MATMODE',
 'PWL_CAPEX',
 'READINESS',
 'SINGLE_MATMODE',
 'SINGLE_PRODMODE',
 'STORAGE',
 'STORAGE_REQ'}

In [61]:
ProcessType.location_level()

{'CREDIT', 'INTERMITTENT', 'STORAGE_DISCHARGE'}

In [62]:
LocationType.location_level()

{'LAND'}

In [39]:
Power.transport

{Grid}

In [46]:
Solar.availability

100

In [40]:
Power.demand

{HO: Theta(Power,HO,demand), NY: 40}

In [ ]:
[((houston, sandiego), CO2), ((houston, sandiego), H2), ((houston, newyork), CO2)]

In [24]:
comp_loc_dict = scenario.resources_locations_discharge

In [33]:
scenario.locations_land

{HO, NY, SD}

In [31]:
scenario.locations_locations_land

{HO: set(), SD: set(), NY: set()}

In [28]:
comp_loc_ordered_set = [(i, j) for i in comp_loc_dict for j in comp_loc_dict[i]]

In [30]:
scenario.resources_locations_store

{CO2(Aq): {HO, SD}, LiI_Power_stored: {HO, NY, SD}}

In [29]:
comp_loc_ordered_set

[(Power, HO),
 (Power, SD),
 (Power, NY),
 (CO2_Vent, HO),
 (CO2_Vent, SD),
 (O2, HO),
 (O2, SD)]

In [26]:
comp_loc_ordered_set

[('Power', 'HO'),
 ('Power', 'SD'),
 ('Power', 'NY'),
 ('CO2_Vent', 'HO'),
 ('CO2_Vent', 'SD'),
 ('O2', 'HO'),
 ('O2', 'SD')]

In [23]:
scenario.resources_locations_discharge

{CO2_Vent: {HO, SD}, Power: {HO, NY, SD}, O2: {HO, SD}}

In [27]:
scenario.resources_discharge

{CO2_Vent, O2, Power}

In [24]:
scenario.processes_locations_expenditure

{AWE: {HO, SD},
 PV: {HO, SD},
 SMRH: {HO, SD},
 LiI: {HO, NY, SD},
 H2FC: {NY},
 WF: {HO, SD},
 AQoff_SMR: {HO, SD}}

In [ ]:
resources_consume_locations = {resource: set() for resource in scenario.resources_consume}
for resource in scenario.resources_consume:
    for location in scenario.locations:
        if hasattr(location, 'resources_consume'):
            if resource in location.resources_consume:
                resources_consume_locations[resource].add(location)

In [ ]:
resources_consume_locations = {resource: {location for location in scenario.locations if hasattr(location, 'resources_consume') and resource in location.resources_consume} for resource in scenario.resources_consume}

In [ ]:
scenario.resources_consume

{CH4, H2O, Solar, Wind}

In [ ]:
resources_consume_locations = {resource: {location for location in self.locations if hasattr(location, 'resources_consume') and resource in location.resources_consume} for resource in self.resources_consume}

In [ ]:
resources_consume_locations 

{Solar: {HO, SD}, CH4: {HO, SD}, Wind: {HO, SD}, H2O: {HO, SD}}

In [ ]:
resources_demand_locations = {resource: {location for location in scenario.locations if hasattr(location, 'resources_demand') and resource in location.resources_demand} for resource in scenario.resources_demand}

In [ ]:
scenario.resource_classifications()

{'CONSUME',
 'DEMAND',
 'DISCHARGE',
 'EMISSION',
 'IMPLICIT',
 'PRODUCE',
 'PURCHASE',
 'SELL',
 'STORE',
 'TRANSPORT'}

In [ ]:
{component: {location for location in getattr(self, 'locations') if hasattr(location, f'{component_set}_{parameter.lower()}') and resource in getattr(location, f'{component_set}_{parameter.lower()}')} for resource in getattr(self, f'{component_set}_{parameter.lower()}')}

In [ ]:
scenario.process_classifications()


{'CREDIT',
 'EMISSION',
 'EXPENDITURE',
 'FAILURE',
 'INTERMITTENT',
 'LAND',
 'LINEAR_CAPEX',
 'MULTI_MATMODE',
 'MULTI_PRODMODE',
 'NO_MATMODE',
 'PWL_CAPEX',
 'READINESS',
 'SINGLE_MATMODE',
 'SINGLE_PRODMODE',
 'STORAGE',
 'STORAGE_DISCHARGE',
 'STORAGE_REQ'}

In [47]:
[((houston.name, sandiego.name), CO2.name), ((houston.name, sandiego.name), H2.name), ((houston.name, newyork.name), CO2.name)]

[(('HO', 'SD'), 'CO2'), (('HO', 'SD'), 'H2'), (('HO', 'NY'), 'CO2')]

In [25]:
help(ResourceParamType)

Help on class ResourceParamType in module energiapy.components.parameters.resource:

class ResourceParamType(enum.Enum)
 |  ResourceParamType(value, names=None, *, module=None, qualname=None, type=None, start=1)
 |  
 |  Resource Parameters
 |  
 |  Method resolution order:
 |      ResourceParamType
 |      enum.Enum
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  CONSUME = <ResourceParamType.CONSUME: 3>
 |  
 |  DEMAND = <ResourceParamType.DEMAND: 5>
 |  
 |  DISCHARGE = <ResourceParamType.DISCHARGE: 4>
 |  
 |  PURCHASE_PRICE = <ResourceParamType.PURCHASE_PRICE: 2>
 |  
 |  SELL_PRICE = <ResourceParamType.SELL_PRICE: 1>
 |  
 |  STORAGE_COST = <ResourceParamType.STORAGE_COST: 9>
 |  
 |  STORE_LOSS = <ResourceParamType.STORE_LOSS: 8>
 |  
 |  STORE_MAX = <ResourceParamType.STORE_MAX: 6>
 |  
 |  STORE_MIN = <ResourceParamType.STORE_MIN: 7>
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited f

In [1]:
from pyomo.environ import *
# from energiapy.utils.scale_utils import scale_pyomo_set


In [23]:
scales1 = TemporalScale(discretization_list=[1, 4], design_scale= 0, scheduling_scale=1)

In [24]:
type(scales1)

energiapy.components.temporal_scale.TemporalScale

In [50]:
scales2 = TemporalScale(discretization_list=[2, 2, 2], design_scale= 0, scheduling_scale=2)


In [51]:
scales1.index_list

[[(0,)], [(0, 0), (0, 1), (0, 2), (0, 3)]]

In [52]:
scales2.index_list

[[(0,), (1,)],
 [(0, 0), (0, 1), (1, 0), (1, 1)],
 [(0, 0, 0),
  (0, 0, 1),
  (0, 1, 0),
  (0, 1, 1),
  (1, 0, 0),
  (1, 0, 1),
  (1, 1, 0),
  (1, 1, 1)]]

In [56]:
m = ConcreteModel()
m.scales = Set(scales1.list,
                    initialize=scales1.scale)
m.scales_scheduling = scale_pyomo_set(
        instance=m, scale_level=scales1.scheduling_scale)
m.scales_design = scale_pyomo_set(
        instance=m, scale_level=scales1.design_scale)



In [111]:
scales.scheduling_scale

1

In [103]:
# m.a = Set(initialize = [(i, j) for i in resources_consume_locations for j in resources_consume_locations[i]])
# m.b = Set(initialize = [(i.name, j.name) for i in scenario.resources_locations_consume for j in scenario.resources_locations_consume[i]])
# m.c = Set(initialize = [((houston.name, sandiego.name), CO2.name), ((houston.name, sandiego.name), H2.name), ((houston.name, newyork.name), CO2.name)])
# m.A = Var(m.a, m.scales_scheduling, within = NonNegativeReals)
# m.B = Var(m.b, m.scales_design, within = NonNegativeReals)
# m.C = Var(m.c, m.scales_scheduling, within = NonNegativeReals)

In [33]:
from pyomo.environ import * 
m = ConcreteModel()

In [104]:
m.d = Set(initialize = [(Train_H2.name, (houston.name, sandiego.name)), (Grid.name, (houston.name, sandiego.name)), (Grid.name, (houston.name, newyork.name))])
m.D = Var(m.d, m.scales_design, within = NonNegativeReals)


In [105]:
m.f = Set(initialize = [Train_H2.name, Grid.name])
m.F = Var(m.f, m.scales_design, within = NonNegativeReals)


In [57]:
m.g = Set(initialize = [(CO2.name, (houston.name, sandiego.name)), (Power.name, (houston.name, sandiego.name)), (H2.name, (houston.name, newyork.name))])

In [58]:
m.G = Var(m.g, m.scales_scheduling, within = NonNegativeReals)

In [59]:
m.G.pprint()

G : Size=12, Index=G_index
    Key                         : Lower : Value : Upper : Fixed : Stale : Domain
      ('CO2', 'HO', 'SD', 0, 0) :     0 :  None :  None : False :  True : NonNegativeReals
      ('CO2', 'HO', 'SD', 0, 1) :     0 :  None :  None : False :  True : NonNegativeReals
      ('CO2', 'HO', 'SD', 0, 2) :     0 :  None :  None : False :  True : NonNegativeReals
      ('CO2', 'HO', 'SD', 0, 3) :     0 :  None :  None : False :  True : NonNegativeReals
       ('H2', 'HO', 'NY', 0, 0) :     0 :  None :  None : False :  True : NonNegativeReals
       ('H2', 'HO', 'NY', 0, 1) :     0 :  None :  None : False :  True : NonNegativeReals
       ('H2', 'HO', 'NY', 0, 2) :     0 :  None :  None : False :  True : NonNegativeReals
       ('H2', 'HO', 'NY', 0, 3) :     0 :  None :  None : False :  True : NonNegativeReals
    ('Power', 'HO', 'SD', 0, 0) :     0 :  None :  None : False :  True : NonNegativeReals
    ('Power', 'HO', 'SD', 0, 1) :     0 :  None :  None : False :  True :

In [ ]:
m = ConcreteModel()


In [50]:
import gurobipy
m = gurobipy.read('example.lp')

Read LP format model from file example.lp
Reading time = 0.01 seconds
: 2 rows, 3 columns, 5 nonzeros


In [55]:
scenario.resources

{CH4,
 CO2,
 CO2(Aq),
 CO2_Vent,
 H2,
 H2O,
 LiI_Power_stored,
 O2,
 Power,
 Solar,
 Wind}

In [53]:
class LPFileCreator:
    def __init__(self, objective, constraints, bounds, file_name="problem.lp"):
        self.objective = objective
        self.constraints = constraints
        self.bounds = bounds
        self.file_name = file_name
    
    def create_lp_file(self):
        content = self._generate_lp_content()
        with open(self.file_name, "w") as file:
            file.write(content)
    
    def _generate_lp_content(self):
        lp_content = []
        
        # Add Objective Function
        lp_content.append("\\* Problem Statement *\\")
        lp_content.append("\nMaximize")
        lp_content.append(f" obj: {self.objective}\n")
        
        # Add Constraints
        lp_content.append("Subject To")
        for i, constraint in enumerate(self.constraints, start=1):
            lp_content.append(f" c{i}: {constraint}")
        lp_content.append("")
        
        # Add Bounds
        lp_content.append("Bounds")
        for bound in self.bounds:
            lp_content.append(f" {bound}")
        lp_content.append("")
        
        # Add End
        lp_content.append("End")
        
        return "\n".join(lp_content)


In [54]:
houston.resources_consume

NameError: name 'houston' is not defined

In [55]:
[f'cons_{i.name}_' for i in houston.resources_consume]

NameError: name 'houston' is not defined

In [56]:
def tuples_to_strings(tuple_list):
    return ['_'.join(map(str, t)) for t in tuple_list]

In [57]:
scales = TemporalScale([1,4], design_scale= 0, scheduling_scale=1)

In [58]:
var = ['cons'] #, 'sell']

In [59]:
res_list = ['res1', 'res2', 'res3']

In [60]:
time_index = tuples_to_strings(scales.index_list[1])

In [61]:
value = {'res1': 5, 'res2': 7, 'res3': 8}

In [62]:
cons = list()

In [63]:
for i,j,k in product(var, res_list, time_index):
    cons.append(f'{i}_{j}_{k} <= {value[j]}')

In [64]:
ooo = {'res1': 7, 'res2': 5, 'res3': 9}

In [65]:
k = [f'{ooo[i]} {i}' for i in res_list]

In [66]:
obj = ' + '.join(k)
obj

'7 res1 + 5 res2 + 9 res3'

In [ ]:
for i,j,k in product(var, res_list, time_index):


In [67]:
bou = [f'0 <= {i}' for i in var]

In [68]:

# Define the problem components
objective = "x + 2 y"
constraints = [
    "x + y >= 1",
    "x <= 2",
    "y <= 2"
]
bounds = [
    "0 <= x",
    "0 <= y"
]

# Create the LP file
# creator = LPFileCreator(objective, constraints, bounds, file_name="example.lp")

creator = LPFileCreator(obj, cons, bou, file_name="example.lp")
creator.create_lp_file()
print("LP file 'example.lp' created successfully.")

LP file 'example.lp' created successfully.


In [69]:
import gurobipy as gb 

a = gb.read('example.lp')

Read LP format model from file example.lp
Reading time = 0.00 seconds
obj: 12 rows, 16 columns, 12 nonzeros


In [71]:
a.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 21.5.0 21F79)

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 12 rows, 16 columns and 12 nonzeros
Model fingerprint: 0x66371886
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 8e+00]
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible or unbounded model


In [ ]:
Maximize
  x + y + z
Subject To
  c0: x + y = 1
  c1: x + 5 y + 2 z <= 10
  qc0: x + y + [ x ^ 2 - 2 x * y + 3 y ^ 2 ] <= 5
Bounds
  0 <= x <= 5
  z >= 2
Generals
  x y z
End


In [60]:
H2O.purchase_price

0.001

In [59]:
H2O.cons_max

In [51]:
m.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 21.5.0 21F79)

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x4d819f46
Model has 1 quadratic constraint
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  QMatrix range    [1e+00, 3e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 5e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [5e+00, 5e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, ga

In [54]:
m.getVars()

[<gurobi.Var x (value 1.0)>,
 <gurobi.Var y (value 0.0)>,
 <gurobi.Var z (value 4.0)>]

In [ ]:
plot_scenario.capacity_factor(scenario = scenario, location= houston, process= PV, fig_size= (9,5), color= 'orange')


![cap fac pv](plots/cf_ho.png)

In [ ]:
plot_scenario.capacity_factor(scenario = scenario, location= sandiego, process= WF, fig_size= (9,5), color= 'blue')


![cap fac wf](plots/wf_sd.png)

In [ ]:
plot_scenario.demand_factor(scenario = scenario, location= newyork, resource= Power, fig_size= (9,5), color= 'red')


![df ny](plots/df_ny.png)

## Optimize for minimum cost

In [ ]:
milp_cost = formulate(scenario= scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT, Constraints.MODE, Constraints.NETWORK},  objective=Objective.COST)

**Ensure no discharge of power in Houston and San Diego**

While the demand for power is zero in these locations, Power is still a dischargeable resource. Hence, the discharge bounds need to be set to zero.
If not set to zero, the system chooses to dispense power instead of establishing storage networks.

In [ ]:

milp_cost.constraint_specific_location_discharge_ho = constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location = houston)
milp_cost.constraint_specific_location_discharge_sd = constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location = sandiego)


In [ ]:

results_cost = solve(scenario = scenario, instance= milp_cost, solver= 'gurobi', name="results_cost", print_solversteps = True)


**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(results=results_cost, source='SD', sink='NY', resource='H2', transport='Train_H2')


![sch t sdny](plots/sch_sdny.png)

In [ ]:
plot_results.transport(results=results_cost, source='HO', sink='NY', resource='H2', transport='Train_H2')

![sch hony](plots/sch_hony.png)

**Cost contribution by technology**

In [ ]:
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.CAPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.VOPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.FOPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.TOTAL, location= 'HO', fig_size= (8,6))

![total cost ho](plots/total_ho.png)

**Location-wise cost contribution by technology**

In [ ]:

plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.CAPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.VOPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.FOPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.TOTAL, fig_size= (8,6))

![total lw](plots/total_lw.png)

**Inventory and production schedules**

In [ ]:
plot_results.schedule(results= results_cost, y_axis= 'Inv', component= 'LiI_Power_stored', location = 'SD', fig_size= (9,5), color = 'steelblue')

![inv lii](plots/lii_inv.png)

In [ ]:
plot_results.schedule(results= results_cost, y_axis= 'P', component= 'PV', location = 'SD', fig_size= (9,5), color = 'steelblue')

![pv p](plots/pv_p.png)

## Optimize to maximize resource discharge 

In [ ]:

milp_demand = formulate(scenario=scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                        Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT, Constraints.MODE, Constraints.NETWORK},  objective=Objective.MAX_DISCHARGE, objective_resource=Power)


In [ ]:

milp_demand.constraint_specific_location_discharge_ho = constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location = houston)
milp_demand.constraint_specific_location_discharge_sd = constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location = sandiego)


In [ ]:

results_demand = solve(scenario=scenario, instance=milp_demand, solver='gurobi',
                       name="results_demand", print_solversteps=True)


**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(results=results_demand, source='SD', sink='NY', resource='H2', transport='Train_H2')


![sch t sdny2](plots/sch_sdny2.png)

In [ ]:
plot_results.transport(results=results_demand, source='HO', sink='NY', resource='H2', transport='Train_H2')

![sch t hony](plots/sch_hony2.png)